In [ ]:

# =============================================================================
# 7. MAIN EXECUTION AND CONFIGURATION
# =============================================================================

def create_mesa_net_config():
    """Create default configuration for MESA-Net"""
    return {
        'model': {
            'input_channels': 15,  # Number of meteorological variables (adjusted)
            'num_layers': 3,
            'hidden_dim': 128,
            'memory_config': MemoryConfig(
                num_states=3,
                hidden_dim=128,
                learning_rates={
                    'alert': 0.1,
                    'normal': 0.01,
                    'suppressed': 0.001
                }
            )
        },
        'training': {
            'batch_size': 16,  # Reduced for memory efficiency
            'learning_rate': 1e-4,
            'num_epochs': 100,
            'sequence_length': 12,  # 3 days of 6h data
            'forecast_horizon': 4,  # 1 day ahead
            'num_workers': 2,  # For data loading
        },
        'data': {
            'zarr_path': "gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr",
            'variables': [
                'total_precipitation_6hr',
                '2m_temperature', '2m_dewpoint_temperature',
                'surface_pressure', 'mean_sea_level_pressure',
                '10m_u_component_of_wind', '10m_v_component_of_wind',
                'u_component_of_wind', 'v_component_of_wind',
                'specific_humidity', 'relative_humidity',
                'total_column_water_vapour', 'total_cloud_cover',
                'vertical_velocity', 'geopotential_at_surface'
            ],
            'time_range': slice("2015", "2023"),  # Recent years for faster development
            'normalize': True
        },
        'loss': {
            'alpha_prediction': 1.0,
            'alpha_state_entropy': 0.1,
            'alpha_transition_smooth': 0.01,
            'alpha_cross_memory': 0.05,
            'alpha_cross_layer': 0.05
        }
    }

def main():
    """Main function to demonstrate MESA-Net usage"""
    
    # Configuration
    config = create_mesa_net_config()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # 1. Initialize data manager and create datasets
    print("Initializing datasets...")
    data_manager = WeatherBench2DataManager(
        zarr_path=config['data']['zarr_path'],
        variables=config['data']['variables'],
        sequence_length=config['training']['sequence_length'],
        forecast_horizon=config['training']['forecast_horizon']
    )
    
    # Create datasets
    train_dataset, val_dataset, test_dataset = data_manager.create_datasets(
        time_range=config['data']['time_range'],
        normalize=config['data']['normalize']
    )
    
    # Create data loaders
    train_loader, val_loader, test_loader = data_manager.create_data_loaders(
        datasets=(train_dataset, val_dataset, test_dataset),
        batch_size=config['training']['batch_size'],
        num_workers=config['training']['num_workers']
    )
    
    print(f"Train samples: {len(train_dataset)}")
    print(f"Val samples: {len(val_dataset)}")
    print(f"Test samples: {len(test_dataset)}")
    
    # 2. Initialize model
    print("Initializing MESA-Net model...")
    model = MESANet(
        input_channels=config['model']['input_channels'],
        num_layers=config['model']['num_layers'],
        hidden_dim=config['model']['hidden_dim'],
        memory_config=config['model']['memory_config']
    ).to(device)
    
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # 3. Initialize loss function and optimizer
    loss_fn = MESANetLoss(
        alpha_prediction=config['loss']['alpha_prediction'],
        alpha_state_entropy=config['loss']['alpha_state_entropy'],
        alpha_transition_smooth=config['loss']['alpha_transition_smooth'],
        alpha_cross_memory=config['loss']['alpha_cross_memory'],
        alpha_cross_layer=config['loss']['alpha_cross_layer']
    )
    
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=config['training']['learning_rate'],
        weight_decay=1e-5
    )
    
    # 4. Initialize trainer
    trainer = MESANetTrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        device=device,
        save_dir="./mesa_net_checkpoints"
    )
    
    # 5. Training
    print("Starting training...")
    trainer.train(num_epochs=config['training']['num_epochs'])
    
    # 6. Evaluation
    print("Starting evaluation...")
    evaluator = MESANetEvaluator(model, device)
    
    # Evaluate model
    print("Evaluating on test set...")
    metrics = evaluator._evaluate_model(model, test_loader)
    print("Final metrics:", metrics)
    
    print("MESA-Net training and evaluation completed!")

# =============================================================================
# 8. QUICK TEST FUNCTION
# =============================================================================

def test_data_loading():
    """Quick test function to verify data loading works"""
    print("Testing data loading...")
    
    config = create_mesa_net_config()
    
    # Create a small test dataset
    try:
        data_manager = WeatherBench2DataManager(
            zarr_path=config['data']['zarr_path'],
            variables=config['data']['variables'][:5],  # Test with fewer variables
            sequence_length=4,  # Shorter sequences for testing
            forecast_horizon=2
        )
        
        train_dataset, val_dataset, test_dataset = data_manager.create_datasets(
            time_range=slice("2023", "2023"),  # Just 2023 for testing
            normalize=True
        )
        
        # Test getting a single sample
        sample_input, sample_target, sample_geo = train_dataset[0]
        
        print(f"✓ Data loading successful!")
        print(f"  Input shape: {sample_input.shape}")
        print(f"  Target shape: {sample_target.shape}")
        print(f"  Geo features shape: {sample_geo.shape}")
        print(f"  Train dataset size: {len(train_dataset)}")
        
        # Test data loader
        train_loader, val_loader, test_loader = data_manager.create_data_loaders(
            datasets=(train_dataset, val_dataset, test_dataset),
            batch_size=2,
            num_workers=0  # No multiprocessing for testing
        )
        
        # Get one batch
        for batch_input, batch_target, batch_geo in train_loader:
            print(f"✓ Batch loading successful!")
            print(f"  Batch input shape: {batch_input.shape}")
            print(f"  Batch target shape: {batch_target.shape}")
            print(f"  Batch geo shape: {batch_geo.shape}")
            break
            
    except Exception as e:
        print(f"✗ Data loading failed: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    # Uncomment for quick data loading test
    # test_data_loading()
    
    # Uncomment for full training
    main()

# =============================================================================
# 9. IMPLEMENTATION PHASES BREAKDOWN
# =============================================================================

"""
PHASE 1: Foundation (Month 1) - UPDATED
├── ✓ WeatherBench2Dataset - Proper PyTorch Dataset class
├── ✓ WeatherBench2DataManager - Dataset and DataLoader creation
├── ✓ Basic memory components - Individual memory types
├── ✓ PredRNN++ baseline - Cross-layer memory flow
└── ✓ Fixed state testing - Verify architecture works

Next Steps for Phase 1:
1. Run test_data_loading() to verify data pipeline
2. Test basic MESANetLayer with fixed states
3. Create simple training script for baseline
4. Verify gradient flow and memory usage
5. Monitor GPU memory consumption

PHASE 2: State Machines (Month 2-3)
├── ✓ StateTransitionNetwork - Attention-based transitions
├── ✓ MemoryStateMachine - State-dependent processing
├── ✓ Cross-memory interactions - Memory coordination
└── ☐ Dynamic state learning - Train state transitions

Next Steps for Phase 2:
1. Implement state transition training
2. Add state regularization
3. Test different attention mechanisms
4. Analyze learned state patterns
5. Validate state interpretability

PHASE 3: Full Integration (Month 3-4)
├── ✓ Complete MESA architecture - All components together
├── ✓ Training pipeline - Full training loop with DataLoader
├── ✓ Evaluation framework - Comprehensive metrics
└── ✓ Interpretability tools - State analysis

Next Steps for Phase 3:
1. Run full training pipeline
2. Test end-to-end training
3. Implement ablation studies
4. Add visualization tools
5. Performance optimization

PHASE 4: Optimization (Month 4-5)
├── ☐ Hyperparameter tuning - Grid search/Bayesian opt
├── ☐ Performance analysis - Speed and accuracy
├── ☐ Ablation studies - Component importance
└── ☐ Publication preparation - Results and writing

Key Implementation Improvements:
1. ✓ Proper PyTorch Dataset class with normalization
2. ✓ Automatic train/val/test splitting
3. ✓ Built-in data loading efficiency
4. ✓ Memory-efficient streaming from cloud
5. ✓ Geographic features handling
6. ✓ Error handling and data validation

Usage:
1. Start with test_data_loading() to verify setup
2. Run main() for full training pipeline
3. Monitor GPU memory and adjust batch_size if needed
4. Use saved checkpoints for evaluation and analysis
"""

class MESANetEvaluator:
    """Evaluation and interpretation tools for MESA-Net"""
    
    def __init__(self, model: MESANet, device: torch.device):
        self.model = model
        self.device = device
        
    def evaluate_precipitation_metrics(self, 
                                     predictions: torch.Tensor,
                                     targets: torch.Tensor) -> Dict[str, float]:
        """Compute precipitation-specific evaluation metrics"""
        predictions = predictions.cpu().numpy()
        targets = targets.cpu().numpy()
        
        # Basic metrics
        mse = np.mean((predictions - targets) ** 2)
        mae = np.mean(np.abs(predictions - targets))
        rmse = np.sqrt(mse)
        
        # Precipitation-specific metrics
        # Critical Success Index (CSI) for precipitation detection
        threshold = 0.1  # mm/6hr precipitation threshold
        pred_binary = (predictions > threshold).astype(int)
        target_binary = (targets > threshold).astype(int)
        
        hits = np.sum((pred_binary == 1) & (target_binary == 1))
        misses = np.sum((pred_binary == 0) & (target_binary == 1))
        false_alarms = np.sum((pred_binary == 1) & (target_binary == 0))
        
        csi = hits / (hits + misses + false_alarms) if (hits + misses + false_alarms) > 0 else 0
        
        # Probability of Detection (POD)
        pod = hits / (hits + misses) if (hits + misses) > 0 else 0
        
        # False Alarm Rate (FAR)
        far = false_alarms / (hits + false_alarms) if (hits + false_alarms) > 0 else 0
        
        return {
            'mse': mse,
            'mae': mae,
            'rmse': rmse,
            'csi': csi,
            'pod': pod,
            'far': far
        }
    
    def analyze_state_patterns(self, states_history: Dict) -> Dict[str, any]:
        """Analyze learned state patterns for interpretability"""
        analysis = {}
        
        # Extract state probabilities over time
        memory_types = ['fast', 'slow', 'spatial', 'spatiotemporal']
        
        for memory_type in memory_types:
            state_evolution = []
            for timestep in states_history['state_probs']:
                state_probs = timestep[memory_type]
                # Average over batch dimension
                avg_probs = torch.mean(state_probs, dim=0).cpu().numpy()
                state_evolution.append(avg_probs)
            
            state_evolution = np.array(state_evolution)  # Shape: (time, num_states)
            
            analysis[f'{memory_type}_state_evolution'] = state_evolution
            analysis[f'{memory_type}_dominant_state'] = np.argmax(state_evolution, axis=1)
            analysis[f'{memory_type}_state_stability'] = np.std(state_evolution, axis=0)
        
        return analysis
    
    def generate_attention_maps(self, 
                               input_sequence: torch.Tensor,
                               geo_features: torch.Tensor) -> Dict[str, torch.Tensor]:
        """Generate attention maps for visualization"""
        self.model.eval()
        
        with torch.no_grad():
            # Forward pass with hooks to capture intermediate attention
            predictions, states_history = self.model(input_sequence, geo_features)
            
            # Extract attention patterns from the last layer
            attention_maps = {}
            
            # This would require modifying the model to return attention weights
            # For now, return placeholder
            batch_size, seq_len, height, width = input_sequence.shape[:2] + input_sequence.shape[-2:]
            
            attention_maps['spatial_attention'] = torch.randn(batch_size, height, width)
            attention_maps['temporal_attention'] = torch.randn(batch_size, seq_len)
            
        return attention_maps
    
    def compare_with_baselines(self, 
                              test_data_loader: WeatherBench2DataLoader,
                              baseline_models: Dict[str, nn.Module]) -> Dict[str, Dict[str, float]]:
        """Compare MESA-Net with baseline models"""
        results = {}
        
        # Evaluate MESA-Net
        mesa_metrics = self._evaluate_model(self.model, test_data_loader)
        results['MESA-Net'] = mesa_metrics
        
        # Evaluate baseline models
        for model_name, model in baseline_models.items():
            baseline_metrics = self._evaluate_model(model, test_data_loader)
            results[model_name] = baseline_metrics
        
        return results
    
    def _evaluate_model(self, model: nn.Module, data_loader: WeatherBench2DataLoader) -> Dict[str, float]:
        """Helper function to evaluate any model"""
        model.eval()
        all_predictions = []
        all_targets = []
        
        with torch.no_grad():
            for batch_idx in range(min(50, len(data_loader) // data_loader.batch_size)):
                start_idx = batch_idx * data_loader.batch_size
                end_idx = start_idx + data_loader.batch_size
                batch_indices = data_loader.time_indices[start_idx:end_idx]
                
                input_seq, target_seq = data_loader.get_sequence_batch(batch_indices)
                input_seq = input_seq.to(self.device)
                target_seq = target_seq.to(self.device)
                
                if hasattr(model, 'forward') and 'geo_features' in model.forward.__code__.co_varnames:
                    # MESA-Net style model
                    batch_size, seq_len, channels, height, width = input_seq.shape
                    geo_features = torch.zeros(batch_size, 4, height, width, device=self.device)
                    predictions, _ = model(input_seq, geo_features)
                else:
                    # Standard model
                    predictions = model(input_seq)
                
                all_predictions.append(predictions.cpu())
                all_targets.append(target_seq.cpu())
        
        all_predictions = torch.cat(all_predictions, dim=0)
        all_targets = torch.cat(all_targets, dim=0)
        
        return self.evaluate_precipitation_metrics(all_predictions, all_targets)

# =============================================================================
# 7. MAIN EXECUTION AND CONFIGURATION
# =============================================================================



# =============================================================================
# 8. IMPLEMENTATION PHASES BREAKDOWN
# =============================================================================

"""
PHASE 1: Foundation (Month 1)
├── ✓ WeatherBench2DataLoader - Stream data from cloud
├── ✓ Basic memory components - Individual memory types
├── ✓ PredRNN++ baseline - Cross-layer memory flow
└── ✓ Fixed state testing - Verify architecture works

Next Steps for Phase 1:
1. Test WeatherBench2DataLoader with actual data
2. Implement _to_tensor() method in data loader
3. Test basic MESANetLayer with fixed states
4. Create simple training script for baseline
5. Verify gradient flow and memory usage

PHASE 2: State Machines (Month 2-3)
├── ✓ StateTransitionNetwork - Attention-based transitions
├── ✓ MemoryStateMachine - State-dependent processing
├── ✓ Cross-memory interactions - Memory coordination
└── ☐ Dynamic state learning - Train state transitions

Next Steps for Phase 2:
1. Implement state transition training
2. Add state regularization
3. Test different attention mechanisms
4. Analyze learned state patterns
5. Validate state interpretability

PHASE 3: Full Integration (Month 3-4)
├── ✓ Complete MESA architecture - All components together
├── ✓ Training pipeline - Full training loop
├── ✓ Evaluation framework - Comprehensive metrics
└── ☐ Interpretability tools - State analysis

Next Steps for Phase 3:
1. Integrate all components
2. Test end-to-end training
3. Implement ablation studies
4. Add visualization tools
5. Performance optimization

PHASE 4: Optimization (Month 4-5)
├── ☐ Hyperparameter tuning - Grid search/Bayesian opt
├── ☐ Performance analysis - Speed and accuracy
├── ☐ Ablation studies - Component importance
└── ☐ Publication preparation - Results and writing

Key Implementation Priorities:
1. Get basic data loading working first
2. Test individual components before integration
3. Start with simple fixed states
4. Add complexity gradually
5. Monitor memory usage and training stability
"""